Copyright 2025 Google LLC.

In [ ]:
# @title Licenciado bajo la Licencia Apache, Versión 2.0 (la "Licencia");
# no puedes usar este archivo excepto en cumplimiento con la Licencia.
# Puedes obtener una copia de la Licencia en
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# A menos que lo requiera la ley aplicable o se acuerde por escrito, el software
# distribuido bajo la Licencia se distribuye "TAL CUAL",
# SIN GARANTÍAS NI CONDICIONES DE NINGÚN TIPO, ya sean expresas o implícitas.
# Consulta la Licencia para conocer el lenguaje específico que rige los permisos y
# limitaciones bajo la Licencia.

# Evaluación y salida estructurada

Bienvenido al curso intensivo de 5 días sobre IA generativa en Kaggle.

En este notebook, aprenderás cómo evaluar modelos generativos y cómo estructurar sus salidas para aplicaciones prácticas.

### Antes de comenzar

Asegúrate de haber configurado tu clave API en los secretos de usuario de Kaggle. Si necesitas ayuda, consulta la [guía de configuración](https://www.kaggle.com/code/markishere/day-1-prompting#Get-started-with-Kaggle-notebooks).

## Configuración inicial

Primero, instala las bibliotecas necesarias y configura tu entorno.

In [2]:
!pip install -Uq "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 6.4 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

genai.__version__

'1.7.0'

### Configura tu clave API

Para ejecutar la siguiente celda, tu clave API debe estar almacenada en un [secreto de Kaggle](https://www.kaggle.com/discussions/product-feedback/114053) llamado `GOOGLE_API_KEY`.

Si aún no tienes una clave API, puedes obtener una en [AI Studio](https://aistudio.google.com/app/apikey). Puedes encontrar [instrucciones detalladas en la documentación](https://ai.google.dev/gemini-api/docs/api-key).

Para hacer que la clave esté disponible a través de los secretos de Kaggle, elige `Secrets` en el menú `Add-ons` y sigue las instrucciones para agregar tu clave o habilitarla para este notebook.

In [4]:
from kaggle_secrets import UserSecretsClient

client = genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY"))

Si recibiste un error del tipo `No user secrets exist for kernel id ...`, entonces necesitas agregar tu clave API a través de `Add-ons`, `Secrets` **y** habilitarla.

![Captura de pantalla de la casilla de verificación para habilitar el secreto GOOGLE_API_KEY](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Reintento automático

Este codelab envía muchas solicitudes, por lo que configura un reintento automático
que asegure que tus solicitudes se reintenten cuando se alcance la cuota por minuto.

In [5]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Evaluación

Cuando uses LLMs en casos del mundo real, es importante entender qué tan bien están funcionando. Las capacidades de generación abierta de los LLMs pueden hacer que muchos casos sean difíciles de medir. En este notebook, recorrerás algunas técnicas simples para evaluar las salidas de los LLMs y entender su rendimiento.

Para este ejemplo, evaluarás una tarea de resumen utilizando el [informe técnico de Gemini 1.5 Pro](https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf). Comienza descargando el PDF al entorno del notebook y subiendo esa copia para usarla con la API de Gemini.

In [6]:
!wget -nv -O gemini.pdf https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf

document_file = client.files.upload(file='gemini.pdf')

2025-03-31 09:30:34 URL:https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf [7228817/7228817] -> "gemini.pdf" [1]


### Resumir un documento

La solicitud de resumen utilizada aquí es bastante básica. Se dirige específicamente al contenido de entrenamiento pero no proporciona orientación adicional.

In [ ]:
request = 'Cuéntame sobre el proceso de entrenamiento utilizado aquí.'

def summarise_doc(request: str) -> str:
  """Ejecuta la solicitud en el documento subido."""
  # Establece la temperatura baja para estabilizar la salida.
  config = types.GenerateContentConfig(temperature=0.0)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = summarise_doc(request)
Markdown(summary)

Certainly! Let's break down the training process used for Gemini 1.5 Pro, based on the information provided in the document.

**Key Aspects of the Training Process:**

1.  **Model Architecture:**
    *   Gemini 1.5 Pro is a sparse Mixture-of-Experts (MoE) Transformer-based model.
    *   It builds upon the research advances of Gemini 1.0 and a longer history of MoE research at Google.
    *   MoE models use a learned routing function to direct inputs to a subset of the model's parameters for processing. This allows for a larger total parameter count while keeping the number of activated parameters constant.

2.  **Training Infrastructure:**
    *   Trained on multiple 4096-chip pods of Google's TPUv4 accelerators.
    *   The training is distributed across multiple datacenters.

3.  **Training Data:**
    *   A variety of multimodal and multilingual data is used.
    *   The pre-training dataset includes data sourced from various domains, including web documents, code, images, audio, and video content.

4.  **Instruction Tuning:**
    *   Gemini 1.5 Pro is fine-tuned on a collection of multimodal data containing paired instructions and appropriate responses.
    *   Further tuning is based on human preference data.

5.  **Long-Context Understanding:**
    *   The model incorporates significant architecture changes to enable long-context understanding of inputs up to 10 million tokens without degrading performance.

6.  **Safety Mitigations:**
    *   Supervised fine-tuning (SFT) and reinforcement learning through human feedback (RLHF) are used to mitigate safety risks.
    *   The safety mitigation is focused on adversarial or "harm-inducing" queries.
    *   New image-to-text SFT data is incorporated, as safety SFT data for text-only queries was not as effective for harm-inducing image-to-text queries.

**In summary:** The training process for Gemini 1.5 Pro involves a combination of a sophisticated MoE Transformer architecture, massive computational resources, diverse multimodal data, instruction tuning, and safety mitigations. The model is designed to handle extremely long contexts, enabling it to process and reason over large amounts of information from various modalities.

### Definir un evaluador

Para una tarea como esta, es posible que desees evaluar varios aspectos, como qué tan bien el modelo siguió la instrucción ("seguimiento de instrucciones"), si incluyó datos relevantes en la instrucción ("fundamentación"), qué tan fácil es leer el texto ("fluidez"), u otros factores como "verbosidad" o "calidad".

Puedes instruir a un LLM para realizar estas tareas de manera similar a como instruirías a un evaluador humano: con una definición clara y una [rúbrica de evaluación](https://es.wikipedia.org/wiki/R%C3%BAbrica_%28educaci%C3%B3n%29).

En este paso, defines un agente de evaluación utilizando un "prompt de resumen" preescrito y lo usas para evaluar la calidad del resumen generado.

Nota: Para más prompts de evaluación preescritos que cubren fundamentación, seguridad, coherencia y más, consulta esta [lista completa de prompts de evaluación de modelos](https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates) de la documentación de Google Cloud.

In [ ]:
import enum

# Define el prompt de evaluación
SUMMARY_PROMPT = """\
# Instrucción
Eres un evaluador experto. Tu tarea es evaluar la calidad de las respuestas generadas por modelos de IA.
Te proporcionaremos la entrada del usuario y las respuestas generadas por la IA.
Debes leer cuidadosamente la entrada del usuario para analizar la tarea y luego evaluar la calidad de las respuestas basándote en los Criterios proporcionados en la sección de Evaluación a continuación.
Asignarás a la respuesta una calificación siguiendo la Rúbrica de Calificación y los Pasos de Evaluación. Da explicaciones paso a paso para tu calificación y solo elige calificaciones de la Rúbrica de Calificación.

# Evaluación
## Definición de Métrica
Evaluarás la calidad del resumen, que mide la capacidad general para resumir texto. Presta especial atención a las restricciones de longitud, como en X palabras o en Y oraciones. La instrucción para realizar una tarea de resumen y el contexto a resumir se proporcionan en el prompt del usuario. La respuesta debe ser más corta que el texto en el contexto. La respuesta no debe contener información que no esté presente en el contexto.

## Criterios
Seguimiento de instrucciones: La respuesta demuestra una comprensión clara de las instrucciones de la tarea de resumen, cumpliendo con todos los requisitos de la instrucción.
Fundamentación: La respuesta contiene información incluida solo en el contexto. La respuesta no hace referencia a ninguna información externa.
Concisión: La respuesta resume los detalles relevantes en el texto original sin una pérdida significativa de información clave, sin ser demasiado verbosa o concisa.
Fluidez: La respuesta está bien organizada y es fácil de leer.

## Rúbrica de Calificación
5: (Muy bueno). El resumen sigue las instrucciones, está fundamentado, es conciso y fluido.
4: (Bueno). El resumen sigue las instrucciones, está fundamentado, es conciso y fluido.
3: (Regular). El resumen sigue mayormente las instrucciones, está fundamentado, pero no es muy conciso ni fluido.
2: (Malo). El resumen está fundamentado, pero no sigue las instrucciones.
1: (Muy malo). El resumen no está fundamentado.

## Pasos de Evaluación
PASO 1: Evalúa la respuesta en aspectos de seguimiento de instrucciones, fundamentación, concisión y fluidez según los criterios.
PASO 2: Califica según la rúbrica.

# Entradas del Usuario y Respuesta Generada por la IA
## Entradas del Usuario

### Instrucción
{prompt}

## Respuesta Generada por la IA
{response}
"""

# Define una clase enum estructurada para capturar el resultado.
class SummaryRating(enum.Enum):
  VERY_GOOD = '5'
  GOOD = '4'
  OK = '3'
  BAD = '2'
  VERY_BAD = '1'


def eval_summary(prompt, ai_response):
  """Evalúa el resumen generado en comparación con el prompt utilizado."""

  chat = client.chats.create(model='gemini-2.0-flash')

  # Genera la respuesta de texto completa.
  response = chat.send_message(
      message=SUMMARY_PROMPT.format(prompt=prompt, response=ai_response)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=SummaryRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


text_eval, struct_eval = eval_summary(prompt=[request, document_file], ai_response=summary)
Markdown(text_eval)

## Evaluation
STEP 1: The AI response describes the training process used for Gemini 1.5 Pro. The response is grounded in the document provided. The response does a good job of describing the key aspects of the training process used.
STEP 2: I think that the response deserves a 4 out of 5. The response is good but it could be more concise.

## Rating
4


En este ejemplo, el modelo generó una justificación textual que se configuró en un contexto de chat. Esta respuesta de texto completo es útil tanto para la interpretación humana como para darle al modelo un lugar para "tomar notas" mientras evalúa el texto y produce una puntuación final. Esta estrategia de "tomar notas" o "pensar" generalmente funciona bien con modelos auto-regresivos, donde el texto generado se pasa de nuevo al modelo en cada paso de generación. Esto significa que las "notas" de trabajo se utilizan al generar la salida final.

En el siguiente turno, el modelo convierte la salida de texto en una respuesta estructurada. Si deseas agregar puntuaciones o usarlas programáticamente, entonces debes evitar analizar la salida de texto no estructurada. Aquí se pasa el esquema `SummaryRating`, por lo que el modelo convierte el historial de chat en una instancia del enum `SummaryRating`.

In [9]:
struct_eval

<SummaryRating.GOOD: '4'>

### Hacer que el prompt de resumen sea mejor o peor

Los modelos de Gemini tienden a ser bastante buenos en tareas como la de resumen directo sin mucho prompting, por lo que deberías esperar ver un resultado como `BUENO` o `MUY BUENO` en la tarea anterior, incluso con un prompt rudimentario. Ejecútalo varias veces para tener una idea de la respuesta promedio.

Para explorar cómo influir en la salida del resumen, considera qué podrías cambiar en el prompt de solicitud de resumen para cambiar el resultado. Echa un vistazo al `SUMMARY_PROMPT` de evaluación para obtener algunas ideas.

Prueba los siguientes ajustes y observa cómo cambian positiva o negativamente el resultado:
* Sé específico con el tamaño del resumen,
* Solicita información específica,
* Pregunta sobre información que no está en el documento,
* Pide diferentes grados de resumen (como "explícalo como si tuviera 5 años" o "con toda la profundidad técnica")

In [ ]:
new_prompt = "Explícame como si tuviera 5 años el proceso de entrenamiento"
# Prueba:
#  ELI5 el proceso de entrenamiento
#  Resume la técnica de evaluación de aguja/pajar en 1 línea
#  Describe la arquitectura del modelo a alguien con un título en ingeniería civil
#  ¿Cuál es el mejor LLM?

if not new_prompt:
  raise ValueError("Intenta establecer un nuevo prompt de resumen.")


def run_and_eval_summary(prompt):
  """Genera y evalúa el resumen utilizando el nuevo prompt."""
  summary = summarise_doc(new_prompt)
  display(Markdown(summary + '\n-----'))

  text, struct = eval_summary([new_prompt, document_file], summary)
  display(Markdown(text + '\n-----'))
  print(struct)

run_and_eval_summary(new_prompt)

Certainly! Here's an ELI5 explanation of the training process for a large language model like Gemini 1.5:

**Imagine you're teaching a puppy to understand and respond to commands.**

1.  **Gathering the Data (The Puppy's Learning Material):**
    *   First, you need lots and lots of examples of text, images, audio, and video. Think of it as a huge library filled with books, pictures, songs, and movies.
    *   This data is used to teach the model about the world and how language works.

2.  **Building the Model (The Puppy's Brain):**
    *   The model is like a big, complicated computer program. It has lots of connections and settings that can be adjusted.
    *   At the beginning, the model doesn't know anything. It's like a puppy with a brand new brain.

3.  **Training the Model (Teaching the Puppy):**
    *   You feed the model the data you gathered earlier.
    *   The model tries to predict what comes next in the data. For example, if you give it the sentence "The cat sat on the...", it might try to guess the next word.
    *   If the model guesses correctly, you give it a little reward (like a treat for the puppy). If it guesses wrong, you give it a little correction.
    *   The model adjusts its connections and settings based on the rewards and corrections. This is how it learns.
    *   This process is repeated millions or billions of times. The model gets better and better at predicting what comes next.

4.  **Fine-Tuning (Polishing the Puppy's Skills):**
    *   Once the model has learned the basics, you can fine-tune it for specific tasks.
    *   For example, you might train it to answer questions, write stories, or translate languages.
    *   This is like teaching the puppy specific tricks, like "sit," "stay," or "fetch."

5.  **Evaluating and Improving (Checking the Puppy's Progress):**
    *   You test the model to see how well it performs on different tasks.
    *   If it makes mistakes, you analyze the mistakes and adjust the training process to improve the model.
    *   This is like taking the puppy to obedience school and working with a trainer to fix any problems.

**Key Ideas:**

*   **Lots of Data:** The more data the model sees, the better it learns.
*   **Prediction:** The model learns by trying to predict what comes next.
*   **Adjustments:** The model adjusts its connections and settings based on feedback.
*   **Iteration:** The training process is repeated many times to improve the model.

In essence, training a large language model is like teaching a very smart puppy to understand and respond to the world around it. The puppy learns by seeing lots of examples, making guesses, and getting feedback. Over time, it becomes very good at understanding and responding to language.

-----

## Evaluation

### Step 1: Assess the response in aspects of instruction following, groundedness, conciseness, and verbosity according to the criteria.
The response is an analogy of the training process for a LLM but the prompt asks to summarize the file. Therefore, the response does not follow the instructions and is not grounded in the document.

### Step 2: Score based on the rubric.
1
-----

SummaryRating.VERY_BAD


## Evaluación en la práctica

La evaluación tiene muchos usos prácticos, por ejemplo:
* Puedes iterar rápidamente en un prompt con un pequeño conjunto de documentos de prueba,
* Puedes comparar diferentes modelos para encontrar el que mejor se adapte a tus necesidades, como encontrar el equilibrio entre precio y rendimiento, o encontrar el mejor rendimiento para una tarea específica.
* Al implementar cambios en un modelo o prompt en un sistema de producción, puedes verificar que el sistema no retroceda en calidad.

En esta sección, probarás dos enfoques de evaluación diferentes.

### Evaluación puntual

La técnica utilizada anteriormente, donde evalúas un solo par de entrada/salida contra algunos criterios, se conoce como evaluación puntual. Esto es útil para evaluar salidas singulares en un sentido absoluto, como "¿fue buena o mala?"

En este ejercicio, probarás diferentes prompts de orientación con un conjunto de preguntas.

In [ ]:
import functools

# Prueba estas instrucciones, o edita y agrega las tuyas.
terse_guidance = "Responde la siguiente pregunta en una sola oración, o lo más cerca posible de eso."
moderate_guidance = "Proporciona una respuesta breve a la siguiente pregunta, usa una cita si es necesario, pero solo lo suficiente para responder la pregunta."
cited_guidance = "Proporciona una respuesta completa y detallada a la siguiente pregunta, citando el documento y proporcionando información adicional tanto como sea posible."
guidance_options = {
    'Terse': terse_guidance,
    'Moderate': moderate_guidance,
    'Cited': cited_guidance,
}

questions = [
    # Descomenta una o más preguntas para probar aquí, o agrega las tuyas.
    # Evaluar más preguntas tomará más tiempo, pero producirá resultados
    # con mayor confianza. En un sistema de producción, puedes tener cientos
    # de preguntas para evaluar un sistema complejo.

    # "¿Qué métrica(s) se utilizan para evaluar el rendimiento de contexto largo?",
    "¿Cómo se desempeña el modelo en tareas de código?",
    "¿Cuántas capas tiene?",
    # "¿Por qué se llama Gemini?",
]

if not questions:
  raise NotImplementedError('¡Agrega algunas preguntas para evaluar!')


@functools.cache
def answer_question(question: str, guidance: str = '') -> str:
  """Genera una respuesta a la pregunta utilizando el documento subido y la orientación."""
  config = types.GenerateContentConfig(
      temperature=0.0,
      system_instruction=guidance,
  )
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[question, document_file],
  )

  return response.text


answer = answer_question(questions[0], terse_guidance)
Markdown(answer)

Gemini 1.5 Pro performs well on code tasks, surpassing Gemini 1.0 Ultra on Natural2Code and showing improvements in coding capabilities compared to previous Gemini models.


Ahora configura un evaluador de preguntas y respuestas, similar a antes, pero utilizando el [prompt de evaluación de calidad de QA puntual](https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates#pointwise_question_answering_quality).

In [ ]:
import enum

QA_PROMPT = """\
# Instrucción
Eres un evaluador experto. Tu tarea es evaluar la calidad de las respuestas generadas por modelos de IA.
Te proporcionaremos la entrada del usuario y las respuestas generadas por la IA.
Debes leer cuidadosamente la entrada del usuario para analizar la tarea y luego evaluar la calidad de las respuestas basándote en las reglas proporcionadas en la sección de Evaluación a continuación.

# Evaluación
## Definición de Métrica
Evaluarás la calidad de la respuesta a la pregunta en el prompt del usuario. Presta especial atención a las restricciones de longitud, como en X palabras o en Y oraciones. La instrucción para realizar una tarea de preguntas y respuestas se proporciona en el prompt del usuario. La respuesta no debe contener información que no esté presente en el contexto (si se proporciona).

Asignarás a la respuesta una puntuación de 5, 4, 3, 2, 1, siguiendo la Rúbrica de Calificación y los Pasos de Evaluación.
Da explicaciones paso a paso para tu puntuación y solo elige puntuaciones de 5, 4, 3, 2, 1.

## Definición de Criterios
Seguimiento de instrucciones: La respuesta demuestra una comprensión clara de las instrucciones de la tarea de preguntas y respuestas, cumpliendo con todos los requisitos de la instrucción.
Fundamentación: La respuesta contiene información incluida solo en el contexto si el contexto está presente en el prompt del usuario. La respuesta no hace referencia a ninguna información externa.
Integridad: La respuesta responde completamente a la pregunta con suficiente detalle.
Fluidez: La respuesta está bien organizada y es fácil de leer.

## Rúbrica de Calificación
5: (Muy bueno). La respuesta sigue las instrucciones, está fundamentada, es completa y fluida.
4: (Bueno). La respuesta sigue las instrucciones, está fundamentada, es completa, pero no es muy fluida.
3: (Regular). La respuesta sigue mayormente las instrucciones, está fundamentada, responde parcialmente a la pregunta y no es muy fluida.
2: (Malo). La respuesta no sigue muy bien las instrucciones, es incompleta o no está completamente fundamentada.
1: (Muy malo). La respuesta no sigue las instrucciones, es incorrecta y no está fundamentada.

## Pasos de Evaluación
PASO 1: Evalúa la respuesta en aspectos de seguimiento de instrucciones, fundamentación, integridad y fluidez según los criterios.
PASO 2: Califica según la rúbrica.

# Entradas del Usuario y Respuesta Generada por la IA
## Entradas del Usuario
### Instrucción
{prompt}

## Respuesta Generada por la IA
{response}
"""

class AnswerRating(enum.Enum):
  VERY_GOOD = '5'
  GOOD = '4'
  OK = '3'
  BAD = '2'
  VERY_BAD = '1'


@functools.cache
def eval_answer(prompt, ai_response, n=1):
  """Evalúa la respuesta generada en comparación con el prompt/pregunta utilizado."""
  chat = client.chats.create(model='gemini-2.0-flash')

  # Genera la respuesta de texto completa.
  response = chat.send_message(
      message=QA_PROMPT.format(prompt=[prompt, document_file], response=ai_response)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=AnswerRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


text_eval, struct_eval = eval_answer(prompt=questions[0], ai_response=answer)
display(Markdown(text_eval))
print(struct_eval)

STEP 1: The response follows the instructions, is grounded, complete, and fluent.
STEP 2: 5. Very good. The answer follows instructions, is grounded, complete, and fluent.

AnswerRating.VERY_GOOD


Ahora ejecuta la tarea de evaluación en un bucle. Ten en cuenta que la instrucción de orientación está oculta para el agente de evaluación. Si pasaras el prompt de orientación, el modelo calificaría en función de si siguió esa orientación, pero para esta tarea el objetivo es encontrar el mejor resultado general basado en la pregunta del usuario, no en la instrucción del desarrollador.

In [ ]:
import collections
import itertools

# Número de veces para repetir cada tarea para reducir el error y calcular un promedio.
# Aumentarlo tomará más tiempo pero dará mejores resultados, prueba con 2 o 3 para empezar.
NUM_ITERATIONS = 1

scores = collections.defaultdict(int)
responses = collections.defaultdict(list)

for question in questions:
  display(Markdown(f'## {question}'))
  for guidance, guide_prompt in guidance_options.items():

    for n in range(NUM_ITERATIONS):
      # Genera una respuesta.
      answer = answer_question(question, guide_prompt)

      # Evalúa la respuesta (ten en cuenta que el prompt de orientación no se pasa).
      written_eval, struct_eval = eval_answer(question, answer, n)
      print(f'{guidance}: {struct_eval}')

      # Guarda la puntuación numérica.
      scores[guidance] += int(struct_eval.value)

      # Guarda las respuestas, en caso de que desees inspeccionarlas.
      responses[(guidance, question)].append((answer, written_eval))


## How does the model perform on code tasks?

Terse: AnswerRating.VERY_GOOD
Moderate: AnswerRating.VERY_GOOD
Cited: AnswerRating.VERY_GOOD


## How many layers does it have?

Terse: AnswerRating.VERY_GOOD
Moderate: AnswerRating.BAD
Cited: AnswerRating.VERY_GOOD


Ahora agrega las puntuaciones para ver cómo se desempeñó cada prompt.

In [14]:
for guidance, score in scores.items():
  avg_score = score / (NUM_ITERATIONS * len(questions))
  nearest = AnswerRating(str(round(avg_score)))
  print(f'{guidance}: {avg_score:.2f} - {nearest.name}')

Terse: 5.00 - VERY_GOOD
Moderate: 3.50 - GOOD
Cited: 5.00 - VERY_GOOD


### Evaluación por pares

El prompt de evaluación puntual utilizado en el paso anterior tiene 5 niveles de calificación en la salida. Esto puede ser demasiado grueso para tu sistema, o tal vez desees mejorar un prompt que ya es "muy bueno".

Otro enfoque para la evaluación es comparar dos salidas entre sí. Esta es la evaluación por pares, y es un paso clave en los algoritmos de clasificación y ordenamiento, lo que te permite usarla para clasificar tus prompts en lugar de, o además del enfoque puntual.

Este paso implementa la evaluación por pares utilizando el [prompt de calidad de QA por pares](https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates#pairwise_question_answering_quality) de la documentación de Google Cloud.

In [ ]:
QA_PAIRWISE_PROMPT = """\
# Instrucción
Eres un evaluador experto. Tu tarea es evaluar la calidad de las respuestas generadas por dos modelos de IA. Te proporcionaremos la entrada del usuario y un par de respuestas generadas por la IA (Respuesta A y Respuesta B). Debes leer cuidadosamente la entrada del usuario para analizar la tarea y luego evaluar la calidad de las respuestas basándote en los Criterios proporcionados en la sección de Evaluación a continuación.

Primero juzgarás las respuestas individualmente, siguiendo la Rúbrica de Calificación y los Pasos de Evaluación. Luego darás explicaciones paso a paso para tu juicio, compararás los resultados para declarar al ganador según la Rúbrica de Calificación y los Pasos de Evaluación.

# Evaluación
## Definición de Métrica
Evaluarás la calidad de la respuesta a la pregunta en el prompt del usuario. Presta especial atención a las restricciones de longitud, como en X palabras o en Y oraciones. La instrucción para realizar una tarea de preguntas y respuestas se proporciona en el prompt del usuario. La respuesta no debe contener información que no esté presente en el contexto (si se proporciona).

## Criterios
Seguimiento de instrucciones: La respuesta demuestra una comprensión clara de las instrucciones de la tarea de preguntas y respuestas, cumpliendo con todos los requisitos de la instrucción.
Fundamentación: La respuesta contiene información incluida solo en el contexto si el contexto está presente en el prompt del usuario. La respuesta no hace referencia a ninguna información externa.
Integridad: La respuesta responde completamente a la pregunta con suficiente detalle.
Fluidez: La respuesta está bien organizada y es fácil de leer.

## Rúbrica de Calificación
"A": La Respuesta A responde a la pregunta dada según los criterios mejor que la respuesta B.
"SAME": La Respuesta A y B responden a la pregunta dada igualmente bien según los criterios.
"B": La Respuesta B responde a la pregunta dada según los criterios mejor que la respuesta A.

## Pasos de Evaluación
PASO 1: Analiza la Respuesta A según los criterios de calidad de preguntas y respuestas: Determina qué tan bien la Respuesta A cumple con los requisitos del usuario, está fundamentada en el contexto, es completa y fluida, y proporciona una evaluación según el criterio.
PASO 2: Analiza la Respuesta B según los criterios de calidad de preguntas y respuestas: Determina qué tan bien la Respuesta B cumple con los requisitos del usuario, está fundamentada en el contexto, es completa y fluida, y proporciona una evaluación según el criterio.
PASO 3: Compara el rendimiento general de la Respuesta A y la Respuesta B según tus análisis y evaluación.
PASO 4: Salida de tu preferencia de "A", "SAME" o "B" en el campo pairwise_choice según la Rúbrica de Calificación.
PASO 5: Salida de tu razonamiento de evaluación en el campo explanation.

# Entradas del Usuario y Respuestas Generadas por la IA
## Entradas del Usuario
### Instrucción
{prompt}

# Respuestas Generadas por la IA

### Respuesta A
{baseline_model_response}

### Respuesta B
{response}
"""


class AnswerComparison(enum.Enum):
  A = 'A'
  SAME = 'SAME'
  B = 'B'


@functools.cache
def eval_pairwise(prompt, response_a, response_b, n=1):
  """Determina cuál de dos respuestas a la misma pregunta es mejor."""

  chat = client.chats.create(model='gemini-2.0-flash')

  # Genera la respuesta de texto completa.
  response = chat.send_message(
      message=QA_PAIRWISE_PROMPT.format(
          prompt=[prompt, document_file],
          baseline_model_response=response_a,
          response=response_b)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=AnswerComparison,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


question = questions[0]
answer_a = answer_question(question, terse_guidance)
answer_b = answer_question(question, cited_guidance)

text_eval, struct_eval = eval_pairwise(
    prompt=question,
    response_a=answer_a,
    response_b=answer_b,
)

display(Markdown(text_eval))
print(struct_eval)

## Pre-evaluation Analysis
Both responses address the prompt question by discussing how the model performs on code tasks. Response B is more comprehensive and provides detailed information from the document, including specific examples and metrics. Therefore, Response B is better.

## Evaluation of Response A
STEP 1: The response provides a brief summary of Gemini 1.5 Pro's performance on code tasks. It mentions that it surpasses Gemini 1.0 Ultra on Natural2Code and shows improvements in coding capabilities.
STEP 2: The response is grounded in the document but lacks specific details.
STEP 3: The response is relatively complete but could benefit from more specific examples and metrics to support its claims.
STEP 4: The response is fluent and easy to understand.

## Evaluation of Response B
STEP 1: The response provides a detailed breakdown of Gemini 1.5 Pro's performance on code tasks. It covers various aspects such as core capabilities, codebase understanding, NLL analysis, HumanEval leakage, Natural2Code, instruction following, and divergence.
STEP 2: The response is grounded in the document and provides specific examples and metrics to support its claims.
STEP 3: The response is comprehensive and provides a thorough overview of the model's performance on code tasks.
STEP 4: The response is well-organized and easy to understand, with clear headings and bullet points.

## Comparison
STEP 1: Both responses address the prompt question, but Response B provides a more detailed and comprehensive answer.
STEP 2: Response B includes specific examples and metrics from the document, while Response A provides a more general summary.
STEP 3: Response B is better organized and easier to follow, with clear headings and bullet points.
STEP 4: Overall, Response B is more informative and helpful.

## Preference
B

## Reasoning
Response B provides a more detailed and comprehensive answer to the prompt question, including specific examples and metrics from the document. It is also better organized and easier to follow. Therefore, Response B is better than Response A.


AnswerComparison.B


Con un evaluador por pares en su lugar, lo único que se requiere para clasificar los prompts entre sí es un comparador.

Este ejemplo implementa los comparadores mínimos requeridos para el orden total (`==` y `<`) y realiza la comparación utilizando `n_iterations` evaluaciones sobre el conjunto de `questions`.

In [ ]:
@functools.total_ordering
class QAGuidancePrompt:
  """Un prompt de orientación para preguntas y respuestas o instrucción del sistema."""

  def __init__(self, prompt, questions, n_comparisons=NUM_ITERATIONS):
    """Crea el prompt. Proporciona preguntas para evaluar y el número de evaluaciones a realizar."""
    self.prompt = prompt
    self.questions = questions
    self.n = n_comparisons

  def __str__(self):
    return self.prompt

  def _compare_all(self, other):
    """Compara dos prompts en todas las preguntas durante n pruebas."""
    results = [self._compare_n(other, q) for q in questions]
    mean = sum(results) / len(results)
    return round(mean)

  def _compare_n(self, other, question):
    """Compara dos prompts en una pregunta durante n pruebas."""
    results = [self._compare(other, question, n) for n in range(self.n)]
    mean = sum(results) / len(results)
    return mean

  def _compare(self, other, question, n=1):
    """Compara dos prompts en una sola pregunta."""
    answer_a = answer_question(question, self.prompt)
    answer_b = answer_question(question, other.prompt)

    _, result = eval_pairwise(
        prompt=question,
        response_a=answer_a,
        response_b=answer_b,
        n=n,  # Cache buster
    )
    # print(f'q[{question}], a[{self.prompt[:20]}...], b[{other.prompt[:20]}...]: {result}')

    # Convierte el enum a los valores de comparación numérica estándar de Python.
    if result is AnswerComparison.A:
      return 1
    elif result is AnswerComparison.B:
      return -1
    else:
      return 0

  def __eq__(self, other):
    """Verificación de igualdad que realiza una evaluación por pares."""
    if not isinstance(other, QAGuidancePrompt):
      return NotImplemented

    return self._compare_all(other) == 0

  def __lt__(self, other):
    """Verificación de orden que realiza una evaluación por pares."""
    if not isinstance(other, QAGuidancePrompt):
      return NotImplemented

    return self._compare_all(other) < 0


Ahora las funciones de ordenamiento de Python "simplemente funcionarán" en cualquier instancia de `QAGuidancePrompt`. Las funciones `answer_question` y `eval_pairwise` están [memoizadas](https://es.wikipedia.org/wiki/Memoizaci%C3%B3n) para evitar generar innecesariamente las mismas respuestas o evaluaciones, por lo que deberías ver que esto se completa rápidamente a menos que hayas cambiado las preguntas, los prompts o el número de iteraciones de los pasos anteriores.

In [ ]:
terse_prompt = QAGuidancePrompt(terse_guidance, questions)
moderate_prompt = QAGuidancePrompt(moderate_guidance, questions)
cited_prompt = QAGuidancePrompt(cited_guidance, questions)

# Ordena en orden inverso, para que el mejor esté primero
sorted_results = sorted([terse_prompt, moderate_prompt, cited_prompt], reverse=True)
for i, p in enumerate(sorted_results):
  if i:
    print('---')

  print(f'#{i+1}: {p}')

#1: Answer the following question in a single sentence, or as close to that as possible.
---
#2: Provide a thorough, detailed answer to the following question, citing the document and supplying additional background information as much as possible.
---
#3: Provide a brief answer to the following question, use a citation if necessary, but only enough to answer the question.


## Desafíos

### Limitaciones de los LLMs

Se sabe que los LLMs tienen problemas en ciertas tareas, y estos desafíos aún persisten al usar LLMs como evaluadores. Por ejemplo, los LLMs pueden tener dificultades para contar el número de caracteres en una palabra (esto es un problema numérico, no un problema de lenguaje), por lo que un evaluador LLM no podrá evaluar con precisión este tipo de tarea. Hay soluciones disponibles en algunos casos, como conectar herramientas para manejar problemas no adecuados para un modelo de lenguaje, pero es importante que entiendas las posibles limitaciones e incluyas evaluadores humanos para calibrar tu sistema de evaluación y determinar una línea base.

Una razón por la que los evaluadores LLM funcionan bien es que toda la información que necesitan está disponible en el contexto de entrada, por lo que el modelo solo necesita atender a esa información para producir el resultado. Al personalizar prompts de evaluación o construir tus propios sistemas, ten esto en cuenta y asegúrate de no depender del "conocimiento interno" del modelo, o de un comportamiento que podría ser mejor proporcionado por una herramienta.

### Mejorando la confianza

Una forma de mejorar la confianza de tus evaluaciones es incluir un conjunto diverso de evaluadores. Es decir, usa los mismos prompts y salidas, pero ejecútalos en diferentes modelos, como Gemini Flash y Pro, o incluso entre diferentes proveedores, como Gemini, Claude, ChatGPT y modelos locales como Gemma o Qwen. Esto sigue la misma idea utilizada anteriormente, donde repetir pruebas para reunir múltiples "opiniones" ayuda a [reducir el error](https://es.wikipedia.org/wiki/Ley_de_los_grandes_n%C3%BAmeros), excepto que al usar diferentes modelos las "opiniones" serán más diversas.


## Aprende más

Para aprender más sobre los sistemas de evaluación, consulta [esta guía](https://cloud.google.com/blog/products/ai-machine-learning/enhancing-llm-quality-and-interpretability-with-the-vertex-gen-ai-evaluation-service?e=48754805) centrada en la evaluación utilizando el Servicio de Evaluación de Gen AI de Google Cloud.

Y asegúrate de leer el **documento técnico adicional** sobre [Evaluación de Modelos de Lenguaje Grandes](https://services.google.com/fh/files/blogs/neurips_evaluation.pdf).

*- [Mark McD](https://linktr.ee/markmcd)*